In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

In [2]:
def float_text(min_v: float, max_v: float, default: float, name: str):
    return widgets.BoundedFloatText(
        value=default,
        min=min_v,
        max=max_v,
        step=0.1,
        description=name,
        disabled=False)

def float_slider(min_v: float, max_v: float, default: float, name: str):
    return widgets.FloatSlider(
        value=default,
        min=min_v,
        max=max_v,
        step=0.01,
        description=name,
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.1f')

In [3]:
from functions import *

In [8]:
def graphics(eps=1.81, s=3, c=4.8, bx1=0, bx2=15, by=1):
    fig, axs = plt.subplots(1, 2, figsize=(25, 12))

    # constants 
    c_ma = find_min_c_approximation(eps, s)
    c_m  = find_min_c(eps, s)
    g_b  = np.log(1 + eps) - 0.001

    # Axis 0: functions from gamma

    # points of plots
    gs = np.linspace(0, g_b, num=100)

    # functions
    exp2 = exponential_expection(eps=eps, shift=s)
    gauss = sub_gaussian(const=c)
    gauss_optimal_approximation = sub_gaussian(const=c_ma)
    gauss_optimal = sub_gaussian(const=c_m)
    
    # plots of functions
    axs[0].plot(gs, exp2(gs), color='red', label=r'$E[exp(\gamma X)]$')
    axs[0].plot(gs, gauss(gs), color='green', label=r'$exp(\frac{c}{2} \gamma^2)$')
    axs[0].plot(gs, gauss_optimal_approximation(gs), color='seagreen', label=r'$exp(\frac{c_{ma}}{2} \gamma^2)$', alpha=0.4)
    axs[0].plot(gs, [1] * len(gs), '--', color='blue', alpha=0.4)

    # points
    g0    = np.log(s / (s + 1) * (1 + eps))
    gb    = np.log(1 + eps) - 0.0001
    gc    = find_intersection(exp2, gauss, (g0, gb))
    g1    = find_intersection(exp2, lambda _: 1, (g0, gc))
    gc_ma = s / c_ma
    gc_m  = s / c_m
    dl     = np.abs(gc_ma - gc_m)

    # plots of points 
    points = [1] * 3
    
    axs[0].plot(gc, gauss(gc), 'o', color='grey')
    
    points[0], = axs[0].plot(gc_ma, gauss_optimal_approximation(gc_ma), 'o', color='grey')
    points[1], = axs[0].plot(g1, 1       , 'o', color='grey')
    points[2], = axs[0].plot(g0, exp2(g0), 'o', color='grey')

    # view settings
    axs[0].set_title(r'X $\sim Geom(1 + eps) - s$',size=16)
    axs[0].set_ylim([0, 4])
    axs[0].set_xlabel(r'$\gamma$',size=15)

    legend_for_points = axs[0].legend(points, 
        [r'$\delta_0$' + ' = ' + f'{round(gc_m,   2)}',
        r'$\gamma_1$' + ' = ' + f'{round(g1,   2)}',
        r'$\gamma_0$' + ' = ' + f'{round(g0,   2)}'], loc=3, fontsize=15)

    axs[0].legend(fontsize=15)
    axs[0].add_artist(legend_for_points)

    # annotations
    axs[0].annotate(r'$\gamma_0$', (g0 + 0.007, exp2(g0) + 0.04), fontsize=15)
    axs[0].annotate(r'$\gamma_1$', (g1 + 0.01, 1 + 0.041), fontsize=15)
    axs[0].annotate(r'$\delta_0$', (gc - 0.033, exp2(gc) + 0.043), fontsize=15)
    axs[0].annotate(r'$\delta_0^{ma}$', (gc_ma - 0.033, gauss_optimal_approximation(gc_ma) + 0.043), fontsize=15)
    axs[0].plot([gc_ma - dl, gc_ma + dl], [gauss_optimal_approximation(gc_ma)]*2, color='b')

    # Axis 1: probabilies

    # points of plots
    ds = np.linspace(0, bx2, num=500)

    # functions
    neg_pr = negative_drift_probability(eps=eps, shift=s)
    neg_pr_ = negative_drift_probability_(eps=eps, shift=s)
    sub_pr = sub_gaussian_probability(eps=eps, shift=s)

    # plots of functions
    axs[1].plot(ds, [neg_pr(d) for d in ds], label=r'$\Pr_{Neg}[T \leq \frac{d}{s}]$')
    axs[1].plot(ds, [neg_pr_(d, g0) for d in ds], alpha=0.5, color='royalblue', label=r'$\Pr_{Neg, \gamma = \gamma_0}[T \leq \frac{d}{s}]$')

    axs[1].plot(ds, [sub_pr(d, c) for d in ds], color='orange', label=r'$\Pr_{Gaus}[T \leq \frac{d}{s}]$')
    axs[1].plot(ds, [sub_pr(d, c_ma) for d in ds], color='yellow', label=r'$\Pr_{Gaus, c=c_{ma}}[T \leq \frac{d}{s}]$')

    # view settings
    axs[1].set_title(r'Negative drift and sub-gaussian probabilities',size=16)
    axs[1].set_ylim([0, by])
    axs[1].set_xlim([bx1, bx2])
    axs[1].set_xlabel(r'$d$ - distance',size=15)
    axs[1].legend(fontsize=15, loc=1)


In [9]:
interactive_plot = interactive(graphics, 
    eps  = float_slider(0, 10, 1.8, "eps"), 
    s    = (0, 5, 0.01), 
    c    = (0, 10, 0.01),
    bx1  = float_text(0, 10000, 0, "x-left:"),
    bx2  = float_text(0, 10000, 15, "x-right:"),
    by   = float_text(0, 10000, 1, "y-top:"))

output = interactive_plot.children[-1]
output.layout.height = '720px'
interactive_plot

interactive(children=(FloatSlider(value=1.8, description='eps', max=10.0, readout_format='.1f', step=0.01), Fl…

In [5]:
from matplotlib.pyplot import figure

def graphics(eps=1.25, s=1.25, cl=7):
    figure(figsize=(8, 6), dpi=80)

    cs = xs = np.linspace(0.2, cl, num=100)

    exp2 = exponential_expection(eps=eps, shift=s)

    g0 = np.log(s / (s + 1) * (1 + eps))
    gb = np.log(1 + eps) - 0.0001
    g1 = find_intersection(exp2, lambda _: 1, (g0, gb))

    def calculate_min(c: float) -> float:
       gauss = sub_gaussian(const=c)
       gc = find_intersection(exp2, gauss, (g0, gb))
       return min(gc, s/c)

    plt.axvline(x=s/np.log(1 + eps - 1 / (3/2 * s)), ymin=0, ymax=1, color='r')
    plt.axvline(x=s/np.log(1 + eps - 2 / ((3/2 * s)**2 * (eps * (3/2 * s - 1) + 2))), ymin=0, ymax=1, color='b', alpha=0.7)
    plt.axvline(x=find_min_c_approximation(eps, s), ymin=0, ymax=1, color='r', alpha=0.5)
    plt.plot(cs, [calculate_min(c) for c in cs])
    plt.plot(cs, [g1 for c in cs])


interactive_plot = interactive(graphics, eps=(0, 10.0, 0.01), s=(0, 5, 0.01), cl=(1, 10, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '400px'
interactive_plot

interactive(children=(FloatSlider(value=1.25, description='eps', max=10.0, step=0.01), FloatSlider(value=1.25,…